# README
## Run on Kaggle
-Make sure to include the following two cells

# Upload the necessary helper files and datasets
-GenerateTrainSequences.Py
-GlobalConstants.py
-HelperFunctions.py

In [1]:
# !pip install wget
# !pip install music21
# !pip install pandas

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Download input files from Google Drive

In [2]:
import wget
import zipfile
import multiprocessing

# def get_inputs( ):
#         # Download the datasets from google drive
#     filename = 'bach-cwg-64l-32t-cmaj-amin.zip' # update this

#     # Update the id, and the filename
#     !wget --no-check-certificate 'https://drive.google.com/file/d/1tmqsN5lZ-YsCsF0lfOLMzNb708NLEIpv' -O bach-cwg-64l-32t-cmaj-amin.zip

#     # Extract to cwd
#     with zipfile.ZipFile(filename,"r") as zip_ref:
#         zip_ref.extractall(".")
    
# p = multiprocessing.Process(target=get_inputs)
# p.start()
# p.join()


# Download model from Google Drive

In [3]:
# #Download a large file from Google Drive
# #https://github.com/nsadawi/Download-Large-File-From-Google-Drive-Using-Python/blob/master/Download-Large-File-from-Google-Drive.ipynb
# #taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
# import requests

# def get_large_file_from_google_drive():

#     def download_file_from_google_drive(id, destination):
#         URL = "https://docs.google.com/uc?export=download"

#         session = requests.Session()

#         response = session.get(URL, params = { 'id' : id }, stream = True)
#         token = get_confirm_token(response)

#         if token:
#             params = { 'id' : id, 'confirm' : token }
#             response = session.get(URL, params = params, stream = True)

#         save_response_content(response, destination)    

#     def get_confirm_token(response):
#         for key, value in response.cookies.items():
#             if key.startswith('download_warning'):
#                 return value

#         return None

#     def save_response_content(response, destination):
#         CHUNK_SIZE = 32768

#         with open(destination, "wb") as f:
#             for chunk in response.iter_content(CHUNK_SIZE):
#                 if chunk: # filter out keep-alive new chunks
#                     f.write(chunk)

#     # Extract the saved model
#     file_id = '1dy6GPj4yzhPmVcylCMCxLu472KWltenn'
#     destination = './model.zip'
#     download_file_from_google_drive(file_id, destination)

#     filename = "model.zip"
#     # Extract to cwd
#     with zipfile.ZipFile(filename,"r") as zip_ref:
#         zip_ref.extractall(".")
        
# p = multiprocessing.Process(target=get_large_file_from_google_drive)
# p.start()
# p.join()

# Imports

In [4]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from GlobalConstants import *
#import pydot
import random
from HelperFunctions import *

In [5]:
# Create Tensorboard

# Clear any logs from previous runs
!rm -rf ./logs/ 
!mkdir ./logs/

# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
        


/bin/sh: 1: wget: not found
unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.6/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [6]:
log_dir = "logs/fit/" + "LOG"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Set some initial parameters and constants

In [7]:
# Update this before running
TRAIN_SEQUENCE_LENGTH=64

INT_SONGS_PATH = os.getcwd()
MAPPING_PATH = os.getcwd()
SAVE_MODEL_PATH = os.getcwd()

S_IDXS = [0, 1, 2, 3, 4, 5, 6]
A_IDXS = [7, 8, 9, 10, 11, 12, 13]
T_IDXS = [14, 15, 16, 17, 18, 19, 20]
B_IDXS = [21, 22, 23, 24, 25, 26, 27]
VOCAB_SIZE = [63, None, 15,10,16, 5, 5]

# Define Generator Functions

In [ ]:
def train_sequence_generator_random(int_songs, 
                                    S_idxs="", 
                                    A_idxs="", 
                                    T_idxs="", 
                                    B_idxs="", 
                                    vocab_size="", 
                                    num_train_sequences=1, 
                                    batch_size=1):
    """Generate a set of train sequences from a current voice and target. 
    This instance of the method randomizes the sequences when training
    """

    # initialize the sequences
  
    S_past = [[] for i in range(len(S_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
    S_future = [[] for i in range(len(S_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
    S_current = [[] for i in range(len(S_idxs))] # (# features, batch_size, )
    
    B_past = [[] for i in range(len(B_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
    B_current = [[] for i in range(len(B_idxs))] # (# features, batch_size, )
    
    A_past = [[] for i in range(len(A_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
    A_current = [[] for i in range(len(A_idxs))] # (# features, batch_size, )
    
    T_past = [[] for i in range(len(T_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
    T_current = [[] for i in range(len(T_idxs))] # (# features, batch_size, )
    
    S_past_oh = []
    S_future_oh=[]
    S_current_oh=[]
    
    B_past_oh = []
    B_current_oh = []
    
    A_past_oh = []
    A_current_oh = []
    
    T_past_oh = []
    T_current_oh = []
    
    batch = 0 # batch counter
    k = 0 # counter for keeping track of where we are in int_songs
    random_indices = random.sample(range(num_train_sequences),num_train_sequences)

    while True:
        i = random_indices[k]
        
        # Create past target melody
        for j, idx in enumerate(B_idxs):
            B_past[j].append(int_songs[idx][i:i+TRAIN_SEQUENCE_LENGTH])
            B_current[j].append(int_songs[idx][i+TRAIN_SEQUENCE_LENGTH])
            
        for j, idx in enumerate(A_idxs):
            A_past[j].append(int_songs[idx][i:i+TRAIN_SEQUENCE_LENGTH])
            A_current[j].append(int_songs[idx][i+TRAIN_SEQUENCE_LENGTH])
            
        for j, idx in enumerate(T_idxs):
            T_past[j].append(int_songs[idx][i:i+TRAIN_SEQUENCE_LENGTH])
            T_current[j].append(int_songs[idx][i+TRAIN_SEQUENCE_LENGTH])
        
        for j, idx in enumerate(S_idxs):
            # Create past current melody:
            S_past[j].append(int_songs[idx][i:i+TRAIN_SEQUENCE_LENGTH])

            # Create current note
            S_current[j].append(int_songs[idx][i+TRAIN_SEQUENCE_LENGTH])

            # Create future current melody
            S_future_forward = int_songs[idx][i+1+TRAIN_SEQUENCE_LENGTH:i+1+2*TRAIN_SEQUENCE_LENGTH]
            S_future[j].append(S_future_forward[::-1]) # reverse the index

        batch+=1

        k += 1
        # increment i and reset it we reach the number of train sequences
        if k == num_train_sequences:
            k = 0
            random_indices = random.sample(range(num_train_sequences),num_train_sequences)

        # yield the batch
        if batch == batch_size:
            
            # one hot encode the vectors
            for j, _ in enumerate(B_idxs):
                if vocab_size[j]!= None:
                    B_past_oh.append(tf.keras.utils.to_categorical(B_past[j], vocab_size[j]))
                    B_current_oh.append(tf.keras.utils.to_categorical(B_current[j], vocab_size[j]))
                else:
                    B_past_oh.append(B_past[j])
                    B_current_oh.append(B_current[j])
                    
            for j, _ in enumerate(A_idxs):
                if vocab_size[j]!= None:
                    A_past_oh.append(tf.keras.utils.to_categorical(A_past[j], vocab_size[j]))
                    A_current_oh.append(tf.keras.utils.to_categorical(A_current[j], vocab_size[j]))
                else:
                    A_past_oh.append(A_past[j])
                    A_current_oh.append(A_current[j])
                    
            for j, _ in enumerate(T_idxs):
                if vocab_size[j]!= None:
                    T_past_oh.append(tf.keras.utils.to_categorical(T_past[j], vocab_size[j]))
                    T_current_oh.append(tf.keras.utils.to_categorical(T_current[j], vocab_size[j]))
                else:
                    T_past_oh.append(T_past[j])
                    T_current_oh.append(T_current[j])
                    
            for j, _ in enumerate(S_idxs):
                if vocab_size[j]!= None:
                    S_past_oh.append(tf.keras.utils.to_categorical(S_past[j], vocab_size[j]))
                    S_current_oh.append(tf.keras.utils.to_categorical(S_current[j], vocab_size[j]))
                    S_future_oh.append(tf.keras.utils.to_categorical(S_future[j], vocab_size[j]))
                else:
                    S_past_oh.append(S_past[j])
                    S_current_oh.append(S_current[j])
                    S_future_oh.append(S_future[j])

            #-------------------------------------------------------------
            B0_0 = np.array(B_past_oh[0]).astype(np.float) # not sure that this code is needed                        
            B2_0 = np.array(B_current_oh[0])
            
            A0_0 = np.array(A_past_oh[0]).astype(np.float) # not sure that this code is needed                        
            A2_0 = np.array(A_current_oh[0])
            
            T0_0 = np.array(T_past_oh[0]).astype(np.float) # not sure that this code is needed                        
            T2_0 = np.array(T_current_oh[0])
            
            S1_0 = np.array(S_past_oh[0]).astype(np.float)
            S2_0 = np.array(S_current_oh[0]).astype(np.float)
            S3_0 = np.array(S_future_oh[0]).astype(np.float)
            S2_6 = np.array(S_current_oh[6]).astype(np.float)

            #------------------------------------------------------------
            yield ((B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6), (B2_0, A2_0, T2_0))

            # re-initialize the sequences
            S_past = [[] for i in range(len(S_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
            S_future = [[] for i in range(len(S_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
            S_current = [[] for i in range(len(S_idxs))] # (# features, batch_size, )

            B_past = [[] for i in range(len(B_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
            B_current = [[] for i in range(len(B_idxs))] # (# features, batch_size, )

            A_past = [[] for i in range(len(A_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
            A_current = [[] for i in range(len(A_idxs))] # (# features, batch_size, )

            T_past = [[] for i in range(len(T_idxs))] # (# features, batch_size, TRAIN_SEQUENCE_lENGTH)
            T_current = [[] for i in range(len(T_idxs))] # (# features, batch_size, )

            S_past_oh = []
            S_future_oh=[]
            S_current_oh=[]

            B_past_oh = []
            B_current_oh = []

            A_past_oh = []
            A_current_oh = []

            T_past_oh = []
            T_current_oh = []

            batch = 0


In [ ]:
# Load the songs as integers
int_songs = open_csv(os.path.join(INT_SONGS_PATH, "int_songs.csv")) # Code to test the train sequence generation

#### Define constants
LOSS = "categorical_crossentropy"

BATCH_SIZE=64*16 # Numbers between 4-64 work the best 

train_split = 0.98

In [ ]:
max_samples = len(int_songs[0])
num_train_samples = int(train_split*max_samples)
int_songs_train = []
int_songs_val = []
for i,feature in enumerate(int_songs):
    int_songs_train.append(int_songs[i][:num_train_samples])
    int_songs_val.append(int_songs[i][num_train_samples:max_samples])
    
NUM_TRAINING_SEQUENCES = len(int_songs_train[0])-2*TRAIN_SEQUENCE_LENGTH
NUM_VALIDATION_SEQUENCES = len(int_songs_val[0])-2*TRAIN_SEQUENCE_LENGTH
#NUM_TRAINING_SEQUENCES = 10000 # FOR TESTING PURPOSES ONLY (COMMENT OUT THE LINE WHEN RUNNING)

TRAINING_STEPS_PER_EPOCH = NUM_TRAINING_SEQUENCES // BATCH_SIZE
VALIDATION_STEPS_PER_EPOCH = NUM_VALIDATION_SEQUENCES // BATCH_SIZE

# Build the model

In [ ]:
LEN_VOCAB = VOCAB_SIZE[0]

# Input Layers
B0_0 = tf.keras.layers.Input(shape=(TRAIN_SEQUENCE_LENGTH,VOCAB_SIZE[0]), name="B0_0")
A0_0 = tf.keras.layers.Input(shape=(TRAIN_SEQUENCE_LENGTH,VOCAB_SIZE[0]), name="A0_0")
T0_0 = tf.keras.layers.Input(shape=(TRAIN_SEQUENCE_LENGTH,VOCAB_SIZE[0]), name="T0_0")
S1_0 = tf.keras.layers.Input(shape=(TRAIN_SEQUENCE_LENGTH,VOCAB_SIZE[0]), name="S1_0")
S2_0 = tf.keras.layers.Input(shape=(VOCAB_SIZE[0]), name="S2_0")
S3_0 = tf.keras.layers.Input(shape=(TRAIN_SEQUENCE_LENGTH,VOCAB_SIZE[0]), name="S3_0")
S2_6 = tf.keras.layers.Input(shape=(VOCAB_SIZE[6]), name="S2_6")

In [ ]:
# # MODEL 2: "LSTMx128x128-Densex512x128"
# MODEL_NAME = "LSTMx128x128-Densex512x128-SYMBOL-SCALE-16"
# LSTM1_SIZE = 256

# DROPOUT_RATE = 0.5

# B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B0_0)
# B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B)
# #B = tf.keras.layers.Dropout(DROPOUT_RATE)(B)

# A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A0_0)
# A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A)
# #A = tf.keras.layers.Dropout(DROPOUT_RATE)(A)

# T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T0_0)
# T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T)
# #T = tf.keras.layers.Dropout(DROPOUT_RATE)(T)

# S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1_0)
# S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1)
# #S1 = tf.keras.layers.Dropout(DROPOUT_RATE)(S1)

# S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3_0)
# S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3)
# #S3 = tf.keras.layers.Dropout(DROPOUT_RATE)(S3)

# d = tf.keras.layers.Concatenate(1)([B,A,T,S1,S3])
# d = tf.keras.layers.Flatten()(d)

# e = tf.keras.layers.Flatten()(S2_0)
# g = tf.keras.layers.Flatten()(S2_6)
# f = tf.keras.layers.Concatenate(1)([d,e,g])

# f = tf.keras.layers.Dense(1024, activation='relu')(f)
# f = tf.keras.layers.Dropout(DROPOUT_RATE)(f)
# f = tf.keras.layers.Dense(1024, activation='relu')(f)

# B2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(f)
# A2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(f)
# T2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(f)

# model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6],outputs=[B2_0, A2_0, T2_0])

In [ ]:
# #MODEL 2: "LSTMx128x128-Densex512x128"
# MODEL_NAME = "LSTMx128x128-Densex512x128-SYMBOL-SCALE-16"
# LSTM1_SIZE = 128

# DROPOUT_RATE = 0.5

# B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B0_0)
# B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B)
# #B = tf.keras.layers.Dropout(DROPOUT_RATE)(B)

# A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A0_0)
# A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A)
# #A = tf.keras.layers.Dropout(DROPOUT_RATE)(A)

# T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T0_0)
# T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T)
# #T = tf.keras.layers.Dropout(DROPOUT_RATE)(T)

# S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1_0)
# S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1)
# #S1 = tf.keras.layers.Dropout(DROPOUT_RATE)(S1)

# S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3_0)
# S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3)
# #S3 = tf.keras.layers.Dropout(DROPOUT_RATE)(S3)

# d = tf.keras.layers.Concatenate(1)([B,A,T,S1,S3])
# d = tf.keras.layers.Flatten()(d)

# e = tf.keras.layers.Flatten()(S2_0)
# g = tf.keras.layers.Flatten()(S2_6)
# h = tf.keras.layers.Concatenate(1)([d,e,g])

# h = tf.keras.layers.Dense(512, activation='relu')(h)
# h = tf.keras.layers.Dropout(DROPOUT_RATE)(h)
# h = tf.keras.layers.Dense(128, activation='relu')(h)

# B2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
# B_flat = tf.keras.layers.Flatten()(B)
# B2_0 = tf.keras.layers.Concatenate(1)([B2_0, B_flat])
# B2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(B2_0)

# A2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
# A_flat = tf.keras.layers.Flatten()(A)
# A2_0 = tf.keras.layers.Concatenate(1)([A2_0, A_flat])
# A2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(A2_0)

# T2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
# T_flat = tf.keras.layers.Flatten()(T)
# T2_0 = tf.keras.layers.Concatenate(1)([T2_0, T_flat])
# T2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(T2_0)

# model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6],outputs=[B2_0, A2_0, T2_0])

In [ ]:
build_from_scratch = 1
MODEL_NAME = "LSTMx128x128-Densex512x128-Densex512x128-SYMBOL-SCALE-64-Dropout-0.2-ALL-SCALES"

if build_from_scratch:
    LSTM1_SIZE = 128
    DENSE1_SIZE = 512
    DENSE2_SIZE = 128

    DROPOUT_RATE = 0.2

    B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B0_0)
    B = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(B)
    #B = tf.keras.layers.Dropout(DROPOUT_RATE)(B)

    A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A0_0)
    A = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(A)
    #A = tf.keras.layers.Dropout(DROPOUT_RATE)(A)

    T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T0_0)
    T = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(T)
    #T = tf.keras.layers.Dropout(DROPOUT_RATE)(T)

    S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1_0)
    S1 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S1)
    #S1 = tf.keras.layers.Dropout(DROPOUT_RATE)(S1)

    S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3_0)
    S3 = tf.keras.layers.LSTM(LSTM1_SIZE, return_sequences=True)(S3)
    #S3 = tf.keras.layers.Dropout(DROPOUT_RATE)(S3)

    d = tf.keras.layers.Concatenate(1)([B,A,T,S1,S3])
    d = tf.keras.layers.Flatten()(d)

    e = tf.keras.layers.Flatten()(S2_0)
    g = tf.keras.layers.Flatten()(S2_6)
    h = tf.keras.layers.Concatenate(1)([d,e,g])

    h = tf.keras.layers.Dense(DENSE1_SIZE, activation='relu')(h)
    h = tf.keras.layers.Dropout(DROPOUT_RATE)(h)
    h = tf.keras.layers.Dense(DENSE2_SIZE, activation='relu')(h)

    B2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
    B_flat = tf.keras.layers.Flatten()(B)
    B2_0 = tf.keras.layers.Concatenate(1)([B2_0, B_flat, S2_0, S2_6])
    B2_0 = tf.keras.layers.Dense(DENSE1_SIZE, activation='relu')(B2_0)
    B2_0 = tf.keras.layers.Dropout(DROPOUT_RATE)(B2_0)
    B2_0 = tf.keras.layers.Dense(DENSE2_SIZE, activation='relu')(B2_0)
    B2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(B2_0)
    B_model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6],outputs=[B2_0])
    B_model._name = "B_model"

    B2_0_1 = tf.keras.layers.Input(shape=(VOCAB_SIZE[0]))
    A2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
    A_flat = tf.keras.layers.Flatten()(A)
    A2_0 = tf.keras.layers.Concatenate(1)([A2_0, A_flat, S2_0, S2_6, B2_0_1])
    A2_0 = tf.keras.layers.Dense(DENSE1_SIZE, activation='relu')(A2_0)
    A2_0 = tf.keras.layers.Dropout(DROPOUT_RATE)(A2_0)
    A2_0 = tf.keras.layers.Dense(DENSE2_SIZE, activation='relu')(A2_0)
    A2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(A2_0)
    A_model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6, B2_0_1],outputs=[A2_0])
    A_model._name = "A_model"

    A2_0_1 = tf.keras.layers.Input(shape=(VOCAB_SIZE[0]))
    T2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='relu')(h)
    T_flat = tf.keras.layers.Flatten()(T)
    T2_0 = tf.keras.layers.Concatenate(1)([T2_0, T_flat, S2_0, S2_6, B2_0_1, A2_0_1])
    T2_0 = tf.keras.layers.Dense(DENSE1_SIZE, activation='relu')(T2_0)
    T2_0 = tf.keras.layers.Dropout(DROPOUT_RATE)(T2_0)
    T2_0 = tf.keras.layers.Dense(DENSE2_SIZE, activation='relu')(T2_0)
    T2_0 = tf.keras.layers.Dense(VOCAB_SIZE[0], activation='softmax')(T2_0)
    T_model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6, B2_0_1, A2_0_1],outputs=[T2_0])
    T_model._name = "T_model"

    B2_0_2 = B_model([B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6])
    A2_0_2 = A_model([B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6, B2_0_2])
    T2_0_2 = T_model([B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6, B2_0_2, A2_0_2])

    model = tf.keras.Model(inputs=[B0_0, A0_0, T0_0, S1_0, S2_0, S3_0, S2_6],outputs=[B2_0_2, A2_0_2, T2_0_2])
    model._name = "SATB_model"

    # Show basic model statistics
    model.summary()
    tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

else:
    model = tf.keras.models.load_model("./model")

# Compile the Model

In [ ]:
print("compiling...")
model.compile(loss=LOSS, 
              optimizer=tf.keras.optimizers.Adam(lr=.001),
              metrics=["Accuracy"])

print("Creatining the train generator...")
# Create the generator for training
train_generator=train_sequence_generator_random(int_songs_train, 
                                                S_idxs=S_IDXS,
                                                A_idxs=A_IDXS,
                                                T_idxs=T_IDXS,
                                                B_idxs=B_IDXS, 
                                                vocab_size=VOCAB_SIZE,                                            
                                                num_train_sequences=NUM_TRAINING_SEQUENCES,
                                                batch_size=BATCH_SIZE)
print("Creatining the val generator...")
# Create the generator for training
val_generator=train_sequence_generator_random(int_songs_val, 
                                                S_idxs=S_IDXS,
                                                A_idxs=A_IDXS,
                                                T_idxs=T_IDXS,
                                                B_idxs=B_IDXS, 
                                                vocab_size=VOCAB_SIZE,                                            
                                                num_train_sequences=NUM_VALIDATION_SEQUENCES,
                                                batch_size=BATCH_SIZE)

# Define some parameters for training
print("Defining training parameters...")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5,
                              patience=3, min_lr=0.0001, min_delta=1e-2,)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="mymodel",
        save_best_only=True,
        monitor="loss",
        verbose=1,
    )

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="loss",
        min_delta=1e-3,
        patience=5,
        verbose=1)


In [ ]:

print("fitting the model...")
# Train the mode
model.fit(x = train_generator,
          steps_per_epoch = TRAINING_STEPS_PER_EPOCH,
          epochs=1,
          verbose=1,
          callbacks = [reduce_lr, 
                       model_checkpoint, 
                       early_stopping,
                       tensorboard_callback
                      ],
        #  validation_data=val_generator,
        #  validation_steps=VALIDATION_STEPS_PER_EPOCH
          )

print("saving the model...")
name = MODEL_NAME + "{}.h5".format(time.time())
model.save(os.path.join(SAVE_MODEL_PATH, name))

# Fit the model

# Plot the model history

In [ ]:
# Print some basic model statistics
history = model.history
print(history.history.values())

plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['accuracy'], loc='upper left')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss'], loc='upper left')
plt.show()

print()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()